**Запуск модели в режиме оффлайн**

In [14]:
#Подключение необходимых библиотек:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils
from tqdm.notebook import tqdm

In [7]:
#Загрузка данных:
emotion_data = pd.read_csv('/content/drive/MyDrive/fer2013.csv')
emotion_data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [8]:
#Разбитие датасета на обучающую и тестовую выборки:
X_train = []
y_train = []
X_test = []
y_test = []

for index, row in emotion_data.iterrows():
  k = row['pixels'].split(" ")
  kint = []
  for i in k:
    kint.append(int(i))
  if row['Usage'] == 'Training':
    X_train.append(np.array(kint))
    y_train.append(row['emotion'])
  elif row['Usage'] == 'PublicTest':
    X_test.append(np.array(kint))
    y_test.append(row['emotion'])

In [9]:
#Преобразование массивов с пикселями в категориальные:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

y_train = np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)

In [15]:
#Формирование нейронной сети:
model = Sequential()
model.add(ZeroPadding2D( (1,1), input_shape=(48,48,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add (ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add (ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add (ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add (ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile (optimizer="sgd", loss='categorical_crossentropy', 
               metrics=['accuracy'])

model.fit(X_train, y_train, batch_size = 40, 
          epochs = 40, verbose = 1, validation_data=(X_test, y_test))

Epoch 1/40
718/718 [==============================] - 37s 51ms/step - loss: 1.8250 - accuracy: 0.2473 - val_loss: 1.8063 - val_accuracy: 0.2497
Epoch 2/40
718/718 [==============================] - 38s 53ms/step - loss: 1.7495 - accuracy: 0.2925 - val_loss: 1.6269 - val_accuracy: 0.3675
Epoch 3/40
718/718 [==============================] - 36s 50ms/step - loss: 1.5779 - accuracy: 0.3863 - val_loss: 1.4841 - val_accuracy: 0.4338
Epoch 4/40
718/718 [==============================] - 39s 54ms/step - loss: 1.4640 - accuracy: 0.4405 - val_loss: 1.4009 - val_accuracy: 0.4661
Epoch 5/40
718/718 [==============================] - 37s 52ms/step - loss: 1.3623 - accuracy: 0.4769 - val_loss: 1.3081 - val_accuracy: 0.4960
Epoch 6/40
718/718 [==============================] - 38s 53ms/step - loss: 1.2822 - accuracy: 0.5131 - val_loss: 1.2969 - val_accuracy: 0.5063
Epoch 7/40
718/718 [==============================] - 37s 52ms/step - loss: 1.2152 - accuracy: 0.5355 - val_loss: 1.2702 - val_accuracy:

In [16]:
#Оценка работы модели: 
loss_and_metrics = model.evaluate(X_test, y_test)
print(loss_and_metrics)

113/113 [==============================] - 2s 14ms/step - loss: 2.4988 - accuracy: 0.5965
[2.498751640319824, 0.5965449810028076]


In [17]:
#Сохранение весов модели:
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
  model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
